# Scratch work

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [40]:
df = pd.read_csv("../data/generated/fet-celr-5000.csv")

In [41]:
start = time.time()

df["cusum"] = df["cusum"].apply(eval)
df["returns"] = df["returns"].apply(eval)
df["drawdowns"] = df["drawdowns"].apply(eval)

print(f"reformatting took: {round((time.time()-start)/60, 2)} minutes")

reformatting took: 6.11 minutes


## About the dataframe:

In [37]:
df.head(3)

,lookback,thres,sell_thres,cusum,returns,drawdowns
0,4000,0.5,-0.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.04338, -0.00092, 0.00689, 0.00409, 0.00307...","[41920140, 83520]"
1,4000,0.5,0.00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.03909, 0.01631, 0.00499, 0.00778, 0.00482,...","[40840200, 562260, 128700, 64080, 57600, 35160..."
2,4000,0.5,0.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.04346, 0.02526, 0.02499, 0.01898, 0.0033, ...","[18416700, 6417000, 2284080, 1861080, 1584240,..."


| Column name | description |
| --- | --- |
| lookback | metric for identifying strategy (ignore) |
| thres | metric for identifying strategy (ignore) |
| sell_thres | metric for identifying strategy (ignore) |
| cusum | cumulative realized returns starting at 0 and updated every minute |
| returns | list of realized returns, where its length == number of trades that took place |
| drawdowns | list of seconds indicating drawdown length, sorted from greatest to least |

Therefore: 
- if you call .cumsum() on `returns`, the graph resembles `cusum` but much smoother because less granular detail. 
- length of `drawdowns` is how many 'dips' the strategy encountered.
- calling .diff() on `cusum` returns `returns` but on a minutely basis. 

In [25]:
def get_wins_and_losses(returns):
    """returns the number of profitable trades and number of losing trades"""
    wins = len(list(filter(lambda x: x>0, returns)))
    losses = len(list(filter(lambda x: x<=0, returns)))
    return wins, losses

def get_average_win_loss(returns):
    """returns the mean % gain of profitable trades and mean % loss of losing trades in decimal format"""
    wins = np.mean(list(filter(lambda x: x>0, returns)))
    losses = np.mean(list(filter(lambda x: x<=0, returns)))
    return wins, losses

def get_winning_losing_streak(returns):
    """returns the length of the winning streak and length of losing streak"""
    w, p_w = 0, 0
    l, p_l = 0, 0
    for trade in returns:
        if trade > 0:
            w += 1
            p_l = l if l>p_l else p_l
            l = 0
        else:
            l += 1
            p_w = w if w>p_w else p_w
            w = 0
    return p_w, p_l

## Below are examples for this trade:

In [26]:
result = df.iloc[5]
result

lookback                                                   2000
thres                                                       0.5
sell_thres                                                  1.0
cusum         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
returns       [-0.04523, 0.01422, 0.0132, 0.02318, 0.10748, ...
drawdowns     [20299140, 4969260, 2049180, 868800, 713400, 7...
Name: 5, dtype: object

### Get number of trades

In [27]:
len(result.returns)

718

### Get # of wins and # of losses

In [28]:
get_wins_and_losses(result.returns)

(537, 181)

### Get mean win amount and mean loss amount (where 0.02 == 2%)

In [29]:
get_average_win_loss(result.returns)

(0.026453780260707636, -0.06609541436464088)

### Get winning and losing streak, where 17 means 17 wins/losses in a row

In [30]:
get_winning_losing_streak(result.returns)

(18, 5)

### Get max drawdown length (in seconds)

In [31]:
result.drawdowns[0] #sorted by longest to shortest

20299140

### Get max drawdown on a trade and max gain on a trade (realized)

In [42]:
min(result.returns), max(result.returns)

(-0.50733, 0.1568)

In [43]:
df["return"] = list(map(lambda x: x[-1], df.cusum))

In [44]:
df.sort_values("return", ascending=False)

,lookback,thres,sell_thres,cusum,returns,drawdowns,return
109,5000,3.00,0.00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.05667, 0.02189, 0.05573, 0.01262, 0.02248, ...","[2288220, 1866300, 1643220, 1553400, 1349760, ...",4.453754
94,5000,2.50,0.00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.02812, 0.04555, 0.02189, 0.04345, 0.00414, ...","[3752760, 2393340, 1704540, 1494420, 1216920, ...",4.452434
93,5000,2.50,-0.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0241, 0.03199, 0.00462, 0.03034, -0.00083, ...","[5308260, 3278340, 2033760, 1629780, 1553880, ...",4.309373
65,5000,1.75,0.00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.0096, 0.05753, 0.02042, 0.06874, 0.01262, ...","[5297400, 3690840, 2592600, 2236740, 2021400, ...",4.306889
108,5000,3.00,-0.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.04314, 0.00462, 0.0421, 0.00766, 0.02248, 0...","[3107820, 2174160, 1953180, 1932960, 1525380, ...",4.235001
...,...,...,...,...,...,...,...
1,5000,0.50,0.00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.03909, 0.01631, 0.00499, 0.01388, 0.0151, ...","[40837620, 147960, 57660, 35940, 34620, 33120,...",-3.131897
61,5000,1.75,-1.50,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.01863, 0.0153, 0.00174, 0.00129, 0.00307, ...","[42047460, 4080, 1800]",-11.371068
35,5000,1.25,-1.00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.03508, -0.00525, -0.00141, -2e-05, -0.0180...",[42074700],-15.321294
11,5000,0.75,-0.50,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.01678, 0.01248, 0.00366, -0.00704, -0.0522...",[42070080],-18.330420


In [39]:
df.sort_values("return", ascending=False)

,lookback,thres,sell_thres,cusum,returns,drawdowns,return
95,4000,2.50,0.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.00123, 0.02317, 0.02189, 0.03611, 0.03606,...","[2312820, 1783320, 1592160, 1581480, 1543920, ...",4.911096
39,4000,1.25,0.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.04346, 0.04344, 0.01702, -0.0088, 0.03334,...","[14919660, 2681400, 1775820, 1503780, 1046400,...",4.805235
15,4000,0.75,0.50,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.03648, 0.04098, 0.0383, 0.01883, 0.01424, ...","[16289520, 4202940, 1732980, 1306740, 854460, ...",4.656443
52,4000,1.50,0.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.04346, 0.05063, 0.03176, 0.00015, 0.07087,...","[8316300, 4227540, 2642100, 1658640, 1503780, ...",4.579836
94,4000,2.50,0.00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0029, 0.00949, 0.00462, 0.03611, 0.02123, 0...","[3882900, 3185220, 2390160, 1698960, 1589640, ...",4.571913
...,...,...,...,...,...,...,...
12,4000,0.75,-0.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.04338, 0.01378, 0.00307, 0.00393, 0.01581,...","[41567880, 186360, 46020, 30600]",-5.121501
61,4000,1.75,-1.50,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.01581, 0.022, -0.04928, -0.02851, 0.00512,...","[42074700, 2280]",-14.488420
35,4000,1.25,-1.00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.01302, 0.00545, 0.00857, 0.0044, 0.01059, ...","[42028680, 7620, 960]",-18.613442
11,4000,0.75,-0.50,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.00487, -0.00704, -0.05226, -0.00491, 0.0051...",[42051840],-23.353953


In [34]:
df.sort_values("return", ascending=False)

,lookback,thres,sell_thres,cusum,returns,drawdowns,return
43,2000,1.25,1.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.04523, 0.02047, 0.02388, 0.02385, 0.10748,...","[14694780, 4967940, 3022440, 1632300, 827760, ...",4.229392
54,2000,1.50,0.75,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.06352, 0.00342, 0.02847, 0.03367, 0.06253,...","[13640760, 4972200, 2661960, 1117860, 743700, ...",4.125456
55,2000,1.50,1.00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.04523, 0.01422, 0.03985, 0.03367, 0.07261,...","[15127320, 4969260, 2749980, 1021440, 731340, ...",4.086052
56,2000,1.50,1.25,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.04523, 0.02047, 0.04466, 0.03367, 0.07261,...","[15351420, 4967940, 2638740, 1426920, 724140, ...",4.023866
41,2000,1.25,0.75,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.06352, 0.00342, 0.00821, 0.02418, 0.09868,...","[13995060, 4972200, 3659880, 1302180, 859920, ...",3.948964
...,...,...,...,...,...,...,...
1,2000,0.50,0.00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.05948, 0.01135, 0.02219, 0.01542, -0.00647...","[41703720, 270120, 19680, 1500]",-16.232218
61,2000,1.75,-1.50,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.03508, -0.02851, 0.01315, 0.00257, 0.00144...",[42074700],-28.621639
35,2000,1.25,-1.00,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.01302, -0.02493, 0.01832, 0.0022, 0.01081,...",[42070140],-37.119296
11,2000,0.75,-0.50,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.00428, 0.01412, -0.02416, -0.00257, -0.000...","[42028680, 960]",-43.624891
